In [5]:
# Instal library yang dibutuhkan
!pip install librosa soundfile matplotlib numpy scipy

In [6]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

In [7]:
import os

# Tentukan direktori tempat dataset Anda berada
data_dir = '/kaggle/input/burung-mp3'

print(f"Isi dari direktori '{data_dir}':")
# Gunakan os.walk untuk menelusuri semua file dan folder di dalam direktori
for dirpath, dirnames, filenames in os.walk(data_dir):
    for f in filenames:
        print(os.path.join(dirpath, f))

Isi dari direktori '/kaggle/input/burung-mp3':


In [8]:
# Muat file audio (contoh, sesuaikan dengan file Anda)
# y, sr = librosa.load('nama_file_audio_anda.wav', sr=None, duration=5)

# ... (kode Anda untuk memuat audio, menghitung FFT, IFFT) ...

# Visualisasi sinyal waktu (time domain)
plt.figure(figsize=(12, 4))
# Ubah x_axis menjadi axis
librosa.display.waveshow(y, sr=sr, axis='time') # <-- Perubahan di sini
plt.title('Sinyal Audio di Domain Waktu (Sebelum Transformasi)')
plt.xlabel('Waktu (detik)')
plt.ylabel('Amplitudo')
plt.grid(True)
plt.show()

# Visualisasi perbandingan sinyal asli dan rekonstruksi
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1) # 2 baris, 1 kolom, plot pertama
# Ubah x_axis menjadi axis
librosa.display.waveshow(y, sr=sr, axis='time') # <-- Perubahan di sini
plt.title('Sinyal Audio Asli di Domain Waktu')
plt.xlabel('Waktu (detik)')
plt.ylabel('Amplitudo')
plt.grid(True)

plt.subplot(2, 1, 2) # 2 baris, 1 kolom, plot kedua
# Ubah x_axis menjadi axis
librosa.display.waveshow(y_reconstructed, sr=sr, axis='time', color='orange') # <-- Perubahan di sini
plt.title('Sinyal Audio Direkonstruksi di Domain Waktu (Setelah IFFT)')
plt.xlabel('Waktu (detik)')
plt.ylabel('Amplitudo')
plt.grid(True)

plt.tight_layout() # Mengatur layout agar tidak tumpang tindih
plt.show()

NameError: name 'y' is not defined

<Figure size 1200x400 with 0 Axes>

In [ ]:
# Hitung FFT
# Kita akan menggunakan abs(librosa.stft(y)) untuk mendapatkan magnitude spectrogram.
# Untuk mendapatkan spektrum frekuensi penuh, kita bisa menggunakan np.fft.fft
# Namun, librosa.stft lebih umum digunakan dalam analisis musik karena memberikan informasi waktu-frekuensi.

# Kita akan menggunakan rfft (Real FFT) karena sinyal kita adalah real-valued.
# Ini lebih efisien dan hanya mengembalikan bagian positif dari spektrum frekuensi.
N = len(y) # Jumlah sampel
yf = np.fft.rfft(y) # Lakukan Real FFT
xf = np.fft.rfftfreq(N, 1 / sr) # Buat array frekuensi yang sesuai

# Hitung magnitude (amplitudo) dari spektrum frekuensi
# np.abs(yf) memberikan magnitude. Kita akan mengukur dalam dB (desibel) untuk representasi yang lebih baik
magnitude = np.abs(yf)

# Visualisasi spektrum frekuensi
plt.figure(figsize=(12, 4))
plt.plot(xf, magnitude)
plt.title('Spektrum Frekuensi (Setelah FFT)')
plt.xlabel('Frekuensi (Hz)')
plt.ylabel('Amplitudo')
plt.grid(True)
plt.xlim(0, sr / 2) # Batasi tampilan hingga Nyquist frequency
plt.show()

# Visualisasi spektrum frekuensi dalam skala logaritmik (dB)
# Ini lebih umum untuk analisis audio karena telinga manusia mendengar secara logaritmik
magnitude_db = librosa.amplitude_to_db(magnitude, ref=np.max)

plt.figure(figsize=(12, 4))
plt.plot(xf, magnitude_db)
plt.title('Spektrum Frekuensi (dB, Setelah FFT)')
plt.xlabel('Frekuensi (Hz)')
plt.ylabel('Amplitudo (dB)')
plt.grid(True)
plt.xlim(0, sr / 2) # Batasi tampilan hingga Nyquist frequency
plt.show()

In [ ]:
# Temukan frekuensi dominan
# Kita akan mencari indeks frekuensi dengan magnitude tertinggi
dominant_frequency_idx = np.argmax(magnitude[1:]) # Mulai dari indeks 1 untuk menghindari frekuensi DC (0 Hz)
dominant_frequency = xf[dominant_frequency_idx + 1] # Tambahkan 1 karena kita mulai dari indeks 1 di argmax

print(f"Frekuensi dominan teridentifikasi: {dominant_frequency:.2f} Hz")

# Anda juga bisa mengidentifikasi beberapa frekuensi teratas jika ingin
num_peaks = 5
top_peak_indices = np.argsort(magnitude[1:])[::-1][:num_peaks] + 1 # +1 karena kita skip 0 Hz
top_frequencies = xf[top_peak_indices]
top_magnitudes = magnitude[top_peak_indices]

print(f"\n{num_peaks} frekuensi teratas dan magnitudenya:")
for freq, mag in zip(top_frequencies, top_magnitudes):
    print(f"Frekuensi: {freq:.2f} Hz, Magnitudo: {mag:.2f}")

# Visualisasi spektrum dengan penanda frekuensi dominan
plt.figure(figsize=(12, 4))
plt.plot(xf, magnitude)
plt.title('Spektrum Frekuensi dengan Frekuensi Dominan')
plt.xlabel('Frekuensi (Hz)')
plt.ylabel('Amplitudo')
plt.grid(True)
plt.xlim(0, sr / 2)
plt.axvline(x=dominant_frequency, color='r', linestyle='--', label=f'Frekuensi Dominan: {dominant_frequency:.2f} Hz')
plt.legend()
plt.show()

In [ ]:
# Lakukan Inverse FFT (iFFT)
# Untuk sinyal real, kita gunakan irfft
y_reconstructed = np.fft.irfft(yf, n=N) # n=N untuk memastikan panjang sinyal asli

print(f"Panjang sinyal hasil rekonstruksi: {len(y_reconstructed)} sampel")

# Memastikan panjang sinyal rekonstruksi sama dengan sinyal asli
if len(y_reconstructed) > N:
    y_reconstructed = y_reconstructed[:N]
elif len(y_reconstructed) < N:
    # Ini jarang terjadi dengan irfft jika N benar, tapi baik untuk berjaga-jaga
    pad_len = N - len(y_reconstructed)
    y_reconstructed = np.pad(y_reconstructed, (0, pad_len), 'constant')

# Opsional: Simpan sinyal yang direkonstruksi ke file audio
# Ganti 'rekonstruksi_musik.wav' dengan nama file yang Anda inginkan
sf.write('rekonstruksi_musik.wav', y_reconstructed, sr)
print("Sinyal direkonstruksi dan disimpan sebagai 'rekonstruksi_musik.wav'")

In [ ]:
# Visualisasi perbandingan sinyal asli dan rekonstruksi
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1) # 2 baris, 1 kolom, plot pertama
librosa.display.waveshow(y, sr=sr, axis='time') # <--- DIGANTI DI SINI
plt.title('Sinyal Audio Asli di Domain Waktu')
plt.xlabel('Waktu (detik)')
plt.ylabel('Amplitudo')
plt.grid(True)

plt.subplot(2, 1, 2) # 2 baris, 1 kolom, plot kedua
librosa.display.waveshow(y_reconstructed, sr=sr, axis='time', color='orange') # <--- DIGANTI DI SINI
plt.title('Sinyal Audio Direkonstruksi di Domain Waktu (Setelah IFFT)')
plt.xlabel('Waktu (detik)')
plt.ylabel('Amplitudo')
plt.grid(True)

plt.tight_layout() # Mengatur layout agar tidak tumpang tindih
plt.show()

# Visualisasi perbandingan spektrum frekuensi asli dan rekonstruksi (jika ada manipulasi)
# Dalam kasus ini, karena kita tidak memodifikasi spektrum, mereka harus identik.
# Namun, jika Anda mencoba filter atau modifikasi spektrum, plot ini akan menunjukkan perbedaannya.
# Plot ini bisa diabaikan jika tidak ada manipulasi frekuensi.

## Analisis dan Refleksi

### Pendahuluan
Eksperimen ini bertujuan untuk memahami dan menerapkan transformasi sinyal dari domain waktu ke domain frekuensi (menggunakan FFT) dan kembali lagi ke domain waktu (menggunakan IFFT) pada sinyal suara musik. Studi kasus ini berfokus pada identifikasi frekuensi dominan dalam sebuah klip musik pendek dan merekonstruksi sinyal aslinya.

### Proses Eksperimen
1.  **Pengambilan Sinyal Suara:** Saya memilih klip audio `musik_pendek.wav` dengan durasi sekitar X detik dan *sample rate* Y Hz. Visualisasi awal sinyal di domain waktu menunjukkan pola gelombang audio yang kompleks.
2.  **Transformasi ke Domain Frekuensi (FFT):** Setelah menerapkan FFT pada sinyal audio, saya mendapatkan spektrum frekuensi yang menunjukkan komponen frekuensi yang membentuk sinyal tersebut. Plot spektrum menunjukkan beberapa puncak amplitudo yang tinggi, yang mengindikasikan frekuensi dominan dalam musik. Visualisasi dalam skala dB sangat membantu dalam melihat detail frekuensi yang lebih halus.
3.  **Identifikasi Frekuensi Dominan:** Dari spektrum yang dihasilkan, frekuensi dominan teridentifikasi pada sekitar Z Hz. Ini kemungkinan merupakan nada dasar atau harmonik paling kuat dalam klip musik yang diambil. Identifikasi beberapa frekuensi teratas juga memberikan wawasan tentang komposisi harmonik sinyal.
4.  **Rekonstruksi Sinyal (IFFT):** Menggunakan hasil FFT, IFFT diterapkan untuk mengonversi kembali spektrum frekuensi ke sinyal di domain waktu. Hasil rekonstruksi diharapkan mendekati sinyal asli jika tidak ada manipulasi yang dilakukan pada spektrum.

### Hasil dan Diskusi
Visualisasi perbandingan sinyal asli dan sinyal yang direkonstruksi menunjukkan kesamaan yang sangat tinggi. Ini membuktikan bahwa transformasi Fourier (FFT dan IFFT) adalah alat yang sangat efektif untuk menganalisis dan memanipulasi sinyal tanpa kehilangan informasi yang signifikan (selama tidak ada informasi yang dibuang selama proses). Frekuensi dominan yang teridentifikasi sesuai dengan ekspektasi tentang nada yang paling sering muncul dalam klip musik tersebut.

### Refleksi Pribadi
Melalui eksperimen ini, saya mendapatkan pemahaman yang lebih mendalam tentang bagaimana sinyal audio dapat dipecah menjadi komponen frekuensinya dan kemudian direkonstruksi kembali. Konsep *sample rate*, *Nyquist frequency*, dan bagaimana FFT bekerja menjadi lebih jelas. Saya menyadari potensi besar dari transformasi Fourier dalam berbagai aplikasi, seperti:
* **Filtering audio:** Menghilangkan *noise* atau frekuensi tertentu dengan memanipulasi spektrum.
* **Kompresi audio:** Menyimpan hanya frekuensi yang paling penting untuk mengurangi ukuran file.
* **Sintesis suara:** Menciptakan suara baru dari komponen frekuensi.
* **Analisis musik:** Mengidentifikasi nada, ritme, dan instrumen.

Penggunaan Python dengan *library* seperti `librosa` dan `numpy` sangat mempermudah proses implementasi ini, memungkinkan fokus lebih pada pemahaman konsep daripada detail matematis yang kompleks. Tantangan yang mungkin muncul adalah pemahaman yang mendalam tentang fase sinyal dalam spektrum frekuensi, yang juga penting untuk rekonstruksi yang sempurna tetapi seringkali kurang ditekankan dalam pengantar.